In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# 1.Preliminares

Primero cargamos las libretas

In [ ]:
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import make_column_transformer,make_column_selector
from sklearn import datasets
from sklearn import tree
from sklearn.metrics import make_scorer,recall_score
# Local application
import utils_2_milla_juan as utils
import utils_milla_juan_1 as utils1
import numpy as np

Fijamos la semilla

In [ ]:
seed=27912
random_state=27912

# 2.Carga de datos y preparacion de estos

## 2.1 Carga de datos

Al igual que en la practica anterior realizamos la carga de datos utilizando el utils y comprobamos que los datos se han cargado correctamente

### Diabetes Indiana

In [ ]:
filepathD="../input/pima-indians-diabetes-database/diabetes.csv"
indexD=None
targetD="Outcome"
dataD=utils.load_data(filepathD,indexD,targetD)

In [ ]:
dataD.sample(5,random_state=seed)


### Cancer Wisconsin

In [ ]:
filepath="../input/breast-cancer-wisconsin-data/data.csv"
indexC="id"
targetC="diagnosis"
dataC=utils.load_data(filepath,indexC,targetC)

In [ ]:
dataC.sample(5,random_state=seed)

### Titanic

In [ ]:
filepath="../input/titanic/train.csv"
targetT="Survived"
indexT="PassengerId"
dataT=utils.load_data(filepath,indexT,targetT)

In [ ]:
dataT.sample(5,random_state=seed)

## 2.2 Obtencion de subconjuntos

Al igual que la practica anterior dividos los subconjuntos entre variables predictorias y de subconjuntos

In [ ]:
(CX,Cy)=utils.divide_dataset(dataC,target="diagnosis")
(DX,Dy)=utils.divide_dataset(dataD,target="Outcome")
(TX,Ty)=utils.divide_dataset(dataT,target="Survived")

Como antes hacemos la prueba con una muestra no sesgada

In [ ]:
CX.sample(5,random_state=seed)

In [ ]:
DX.sample(5,random_state=seed)

In [ ]:
TX.sample(5,random_state=seed)

In [ ]:
Cy.sample(5,random_state=seed)

In [ ]:
Dy.sample(5,random_state=seed)

In [ ]:
Ty.sample(5,random_state=seed)

## 2.3 Holdout

Ahora deberemos de hacer un holdout para separar las instancias de test y de train y utilizaremos una muestra de entrenamiento de un 70%.

Definimos el train_size

In [ ]:
train_size=0.7

In [ ]:
(DX_train,DX_test,Dy_train,Dy_test)=train_test_split(DX,Dy,stratify=Dy,random_state=seed,train_size=train_size)
(CX_train,CX_test,Cy_train,Cy_test)=train_test_split(CX,Cy,stratify=Cy,random_state=seed,train_size=train_size)
(TX_train,TX_test,Ty_train,Ty_test)=train_test_split(TX,Ty,stratify=Ty,random_state=seed,train_size=train_size)

Comprobamos con muestras no sesgadas que ha salido todo bien(como se puede observar los "_train" seran aquellos que se utilizaran para entrenar y los "_test" para hacer el test).


In [ ]:
DX_train.sample(5,random_state=seed)

In [ ]:
CX_train.sample(5,random_state=seed)


In [ ]:
TX_train.sample(5,random_state=seed)

In [ ]:
Dy_train.sample(5,random_state=seed)

In [ ]:
Cy_train.sample(5,random_state=seed)

In [ ]:
Ty_train.sample(5,random_state=seed)

In [ ]:
DX_test.sample(5,random_state=seed)

In [ ]:
CX_test.sample(5,random_state=seed)

In [ ]:
TX_test.sample(5,random_state=seed)

In [ ]:
Dy_test.sample(5,random_state=seed)

In [ ]:
Cy_test.sample(5,random_state=seed)

In [ ]:
Ty_test.sample(5,random_state=seed)

In [ ]:
dataC_train=utils.join_dataset(CX_train,Cy_train)
dataD_train=utils.join_dataset(DX_train,Dy_train)
dataT_train=utils.join_dataset(TX_train,Ty_train)

### Preparacion de pipelines

In [ ]:
copia1=dataC_train.copy()
correC=utils1.sobrecor(copia1,"diagnosis")
correC.append("Unnamed: 32")
p2C=utils1.QuitarColumnas(correC)
copia1=utils1.eliminarColumnas(copia1,correC).copy()
listaRangoC=utils1.fuerarango3(copia1,"diagnosis")
p3C=utils1.CambiarPorNullFR(listaRangoC,"pp")
copia1=utils1.cambiarporNull(copia1,listaRangoC,"pp").copy()
mediaC=copia1.mean()
p4C=utils1.NulltoMean(mediaC,0)

In [ ]:
copia2=dataD_train.copy()
CAD=utils1.ColumnasAnomalas(copia2,["Pregnancies","Age","DiabetesPedigreeFunction","Outcome"],20)
p1D=utils1.QuitarColumnas(['SkinThickness', 'Insulin'])
copia2=utils1.eliminarColumnas(copia2,CAD).copy()
ListaRangoD=utils1.fuerarango3(copia2,"Outcome")
ListaRangoD[1][0]=0
ListaRangoD[1][4]=0
ListaRangoD[2][4]=2.5
p2D=utils1.CambiarPorNullFR(ListaRangoD,0)
p3D=utils1.ZerotoNone(["Pregnancies","Age","DiabetesPedigreeFunction"])
copia2=utils1.QuitarAnomalos(copia2,["Pregnancies","Age","DiabetesPedigreeFunction","Outcome"]).copy()
mediaD=copia2.mean()
p4D=utils1.NulltoMean(mediaD,0)
copia2=utils1.NonetoMean(copia2,mediaD,"Outcome").copy()

In [ ]:
copia3=dataT_train.copy()
p1T=utils1.QuitarColumnas(["Name","Ticket"])
copia3=utils1.eliminarColumnas(copia3,["Name","Ticket"])
copia3=utils1.cambiarANum(copia3,"Sex",["female","male"],[0,1]).copy()
copia3=utils1.cambiarANum(copia3,"Embarked",["C","Q","S"],[0,1,2]).copy()
p2T=utils1.cambiarCharInt("Sex",["female","male"],[0,1])
p3T=utils1.cambiarCharInt("Embarked",["C","Q","S"],[0,1,2])
listaTR=utils1.fuerarango3(copia3,["Survived","Sex","Embarked","Cabin"])
p4T=utils1.CambiarPorNullFR(listaTR,"pp")
copia3=utils1.cambiarporNull(copia3,listaTR,"pp").copy()
p5T=utils1.QuitarColumnas(["Age","Cabin","SibSp","Fare"])
copia3=utils1.eliminarColumnas(copia3,["Age","Cabin","SibSp","Fare"]).copy()
mediaT=copia3.mean()
p6T=utils1.NulltoMean(mediaT,0)

# 3.Modelos de clasificacion supervisada

## 3.1 Vecinos mas cercanos

In [ ]:
n_neighbors=10
k_neighbors_model=KNeighborsClassifier(n_neighbors)


## 3.2 Arboles de decision

In [ ]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)
aux=decision_tree_model
aux.fit(CX_train.drop(columns="Unnamed: 32"),Cy_train)
#print(tree.export_text(aux))
fig = plt.figure(figsize=(20,20))
_=tree.plot_tree(aux, 
                   feature_names=CX_train.columns,  
                   class_names=targetC,
                   filled=True)
plt.show()

## 3.3 Adaptive Boosting

In [ ]:
adaboost_model=AdaBoostClassifier(random_state=random_state)

## 3.4Bootstrap Aggregating

In [ ]:
bagging_model=BaggingClassifier(random_state=random_state)

## 3.5 Ramddom Forest

In [ ]:
random_forest_model=RandomForestClassifier(random_state=random_state)

## 3.6 Gradient Tree Bosting

In [ ]:
gradient_boosting_model=GradientBoostingClassifier(random_state=random_state)

## 3.7 Histogram-Based Gradient Boosting

In [ ]:
hist_gradient_boosting_model=HistGradientBoostingClassifier(random_state=random_state)

# 4.Evaluacion de modelos

In [ ]:
n_splits=10
n_repeats=5
cv=RepeatedStratifiedKFold(n_splits=n_splits,
                          n_repeats=n_repeats,
                          random_state=random_state)

### 4.1.1 Vecinos mas cercanos

In [ ]:
model_neighborsC=k_neighbors_model
pipeline_neighborsC=make_pipeline(p2C,p3C,p4C,model_neighborsC)
utils.evaluate_estimator(pipeline_neighborsC,CX_train,Cy_train,cv,scoring='precision')

#### Diabetes

In [ ]:
model_neighborsD=k_neighbors_model
pipeline_neighborsD=make_pipeline(p1D,p2D,p3D,p4D,model_neighborsD)
utils.evaluate_estimator(pipeline_neighborsD,DX_train,Dy_train,cv,scoring='precision')

#### Titanic

In [ ]:
model_neighborsT=k_neighbors_model
pipeline_neighborsT=make_pipeline(p1T,p2T,p3T,p4T,p5T,p6T,model_neighborsT)
utils.evaluate_estimator(pipeline_neighborsT,TX_train,Ty_train,cv,scoring='precision')

### 4.1.2 Arboles de decision

In [ ]:
model_treeC=decision_tree_model
pipeline_treeC=make_pipeline(p2C,p3C,p4C,model_treeC)
utils.evaluate_estimator(pipeline_treeC,CX_train,Cy_train,cv,scoring='precision')

#### Diabetes

In [ ]:
model_treeD=decision_tree_model
pipeline_treeD=make_pipeline(p1D,p2D,p3D,p4D,model_treeD)
utils.evaluate_estimator(pipeline_treeD,DX_train,Dy_train,cv,scoring='precision')

#### Titanic

In [ ]:
model_treeT=decision_tree_model
pipeline_treeT=make_pipeline(p1T,p2T,p3T,p4T,p5T,p6T,model_treeT)
utils.evaluate_estimator(pipeline_treeT,TX_train,Ty_train,cv,scoring='precision')

### 4.1.3 Adaptive boosting

In [ ]:
model_adaboostC=adaboost_model
pipeline_adaboostC=make_pipeline(p2C,p3C,p4C,model_adaboostC)
utils.evaluate_estimator(pipeline_adaboostC,CX_train,Cy_train,cv,scoring='precision')

#### Diabetes

In [ ]:
model_adaboostD=adaboost_model
pipeline_adaboostD=make_pipeline(p1D,p2D,p3D,p4D,model_adaboostC)
utils.evaluate_estimator(pipeline_adaboostD,DX_train,Dy_train,cv,scoring='precision')

Titanic

In [ ]:
model_adaboostT=adaboost_model
pipeline_adaboostT=make_pipeline(p1T,p2T,p3T,p4T,p5T,p6T,model_adaboostT)
utils.evaluate_estimator(pipeline_adaboostT,TX_train,Ty_train,cv,scoring='precision')

### 4.1.4 Bootstrap Aggregating

In [ ]:
model_baggingC=bagging_model
pipeline_baggingC=make_pipeline(p2C,p3C,p4C,model_baggingC)
utils.evaluate_estimator(pipeline_baggingC,CX_train,Cy_train,cv,scoring='precision')

#### Diabetes

In [ ]:
model_baggingD=bagging_model
pipeline_baggingD=make_pipeline(p1D,p2D,p3D,p4D,model_baggingC)
utils.evaluate_estimator(pipeline_baggingD,DX_train,Dy_train,cv,scoring='precision')

#### Titanic

In [ ]:
model_baggingT=bagging_model
pipeline_baggingT=make_pipeline(p1T,p2T,p3T,p4T,p5T,p6T,model_baggingT)
utils.evaluate_estimator(pipeline_baggingT,TX_train,Ty_train,cv,scoring='precision')

### 4.1.5 Random Forests

In [ ]:
model_rfC=random_forest_model
pipeline_rfC=make_pipeline(p2C,p3C,p4C,model_rfC)
utils.evaluate_estimator(pipeline_rfC,CX_train,Cy_train,cv,scoring='precision')

#### Diabetes

In [ ]:
model_rfD=random_forest_model
pipeline_rfD=make_pipeline(p1D,p2D,p3D,p4D,model_rfD)
utils.evaluate_estimator(pipeline_rfD,DX_train,Dy_train,cv,scoring='precision')

#### Titanic

In [ ]:
model_rfT=random_forest_model
pipeline_rfT=make_pipeline(p1T,p2T,p3T,p4T,p5T,p6T,model_rfT)
utils.evaluate_estimator(pipeline_rfT,TX_train,Ty_train,cv,scoring='precision')

### 4.1.6 Gradient Tree Boosting

In [ ]:
model_gradientC=gradient_boosting_model
pipeline_gradientC=make_pipeline(p2C,p3C,p4C,model_gradientC)
utils.evaluate_estimator(pipeline_gradientC,CX_train,Cy_train,cv,scoring='precision')

#### Diabetes

In [ ]:
model_gradientD=gradient_boosting_model
pipeline_gradientD=make_pipeline(p1D,p2D,p3D,p4D,model_gradientD)
utils.evaluate_estimator(pipeline_gradientD,DX_train,Dy_train,cv,scoring='precision')

#### Titanic

In [ ]:
model_gradientT=gradient_boosting_model
pipeline_gradientT=make_pipeline(p1T,p2T,p3T,p4T,p5T,p6T,model_gradientT)
utils.evaluate_estimator(pipeline_gradientT,TX_train,Ty_train,cv,scoring='precision')

### 4.1.7 Histogram-Based Gradient Boosting

#### Cancer Wilsconsin

In [ ]:
model_h_gradientC=hist_gradient_boosting_model
pipeline_h_gradientC=make_pipeline(p2C,p3C,p4C,model_h_gradientC)
utils.evaluate_estimator(pipeline_h_gradientC,CX_train,Cy_train,cv,scoring='precision')

#### Diabetes

In [ ]:
model_h_gradientD=hist_gradient_boosting_model
pipeline_h_gradientD=make_pipeline(p1D,p2D,p3D,p4D,model_h_gradientD)
utils.evaluate_estimator(pipeline_h_gradientD,DX_train,Dy_train,cv,scoring='precision')

#### Titanic

In [ ]:
model_h_gradientT=hist_gradient_boosting_model
pipeline_h_gradientT=make_pipeline(p1T,p2T,p3T,p4T,p5T,p6T,model_h_gradientT)
utils.evaluate_estimator(pipeline_h_gradientT,TX_train,Ty_train,cv,scoring='precision')

# 5.Seleccion de modelos

 Para el scoring en los tres utilizaremos recall_score y roc_auc 

In [ ]:
#scoring=['recall','precision']
scoring = {'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label=1)}
refit='recall'

Lo primero que haremos sera crear los distintos pasos de pipeline


In [ ]:
pasosC=[('p2C',p2C),('p3C',p3C),('p4C',p4C)]
pasosD=[('p1d',p1D),('p2d',p2D),('p3d',p3D),('p4d',p4D)]
pasosT=[('p1t',p1T),('p2t',p2T),('p3t',p3T),('p4t',p4T),('p5t',p5T),('p6t',p6T)]

In [ ]:
def optimize_params3(estimator,
                    X, y, cv,
                    scoring=None,
                    **param_grid):
    """Exhaustive search over specified parameter values for an estimator."""
    n_jobs = -1
    verbose = 1
    # El refit sera la media de las metricas
    def refit_aux(cv_results):
        cv_results['mean']=(cv_results['mean_test_AUC'] + cv_results['mean_test_Recall'])/2
        return cv_results['mean'].argmax()
    # Return the train scores to measure the impact
    # of the overfitting and underfitting trade-off
    return_train_score = True

    grid_search = utils.GridSearchCV(estimator,
                               param_grid,
                               scoring=scoring,
                               n_jobs=n_jobs,
                               cv=cv,
                               refit=refit_aux,
                               verbose=verbose,
                               return_train_score=return_train_score)

    grid_search = grid_search.fit(X, y)

    best_params = grid_search.best_params_

    cv_results = grid_search.cv_results_
    cv_results = pd.DataFrame(cv_results)

    # Drop the results for each split to make it easier
    # the visualization of the cross-validation results
    regex = cv_results.filter(regex="split")
    cv_results = cv_results.drop(regex, axis=1)
    cv_results = cv_results.sort_values(by="mean")

    display(cv_results)
    display(best_params)

    return grid_search
def optimize_params4(estimator,
                    X, y, cv,
                    scoring=None,
                    **param_grid):
    """Exhaustive search over specified parameter values for an estimator."""
    n_jobs = -1
    verbose = 1
    # El refit sera la media de las metricas aqui esto no lo usamos(aun)
    def refit_aux(cv_results):
        cv_results['mean']=(cv_results['mean_test_AUC'] + cv_results['mean_test_Recall'])/2
        return cv_results['mean'].argmax()
    # Return the train scores to measure the impact
    # of the overfitting and underfitting trade-off
    return_train_score = True

    grid_search = utils.GridSearchCV(estimator,
                               param_grid,
                               scoring=scoring,
                               n_jobs=n_jobs,
                               cv=cv,
                               refit=refit_aux,
                               verbose=verbose,
                               return_train_score=return_train_score)

    grid_search = grid_search.fit(X, y)

    best_params = grid_search.best_params_

    cv_results = grid_search.cv_results_
    cv_results = pd.DataFrame(cv_results)

    # Drop the results for each split to make it easier
    # the visualization of the cross-validation results
    regex = cv_results.filter(regex="split")
    cv_results = cv_results.drop(regex, axis=1)
    #cv_results = cv_results.sort_values(by="mean")

    display(cv_results)
    display(best_params)

    return grid_search

## 5.1 Vecinos más cercanos

### 5.1.1 Cancer Wisconsin

Empezamos con el de vecinos mas cercanos que busca aquellas tuplas que mas se asimilan a la que se quiere predecir 

In [ ]:
estimator=k_neighbors_model
aux=pasosC.copy()
aux2=utils.Normalizar()
#aux.append(aux2)
aux.append(('knn',estimator))
CYT=Cy_train.replace({"B":1,"M":0}).copy()
pipe=Pipeline(steps=aux)
weights=["uniform","distance"]
n_neighbors=np.arange(5,25,2)
algorith=["auto","brute"]
#scoring = (make_scorer(recall_score, pos_label="1"))
k_neighbors_clfC = optimize_params3(pipe,
                                       CX_train,CYT, cv,
                                        scoring=scoring, 
                                        knn__weights=weights,
                                        knn__n_neighbors=n_neighbors,
                                        knn__algorithm=algorith
                                       )


In [ ]:
estimator=k_neighbors_model
aux=pasosD.copy()
aux.append(('knn',estimator))
pipe=Pipeline(steps=aux)
weights=["uniform","distance"]
n_neighbors=np.arange(5,25,2)
scoring = {'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label=1)}
DXn=utils.normalizar(DX_train).copy()
scoring2 = {'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label=1)}
k_neighbors_clfD = optimize_params3(pipe,
                                        DXn,Dy_train, cv,
                                        scoring=scoring,
                                        knn__weights=weights,
                                        knn__n_neighbors=n_neighbors,
                                        knn__algorithm=algorith
                                        )

In [ ]:
estimator=k_neighbors_model
aux=pasosT.copy()
aux.append(('knn',estimator))
pipe=Pipeline(steps=aux)
weights=["uniform","distance"]
metric=["euclidean", "manhattan", "minkowski"]
n_neighbors=np.arange(5,20,2)
k_neighbors_clfT = optimize_params3(pipe,
                                       TX_train,Ty_train, cv,
                                        scoring=scoring, 
                                        knn__weights=weights,
                                        knn__n_neighbors=n_neighbors,
                                        knn__algorithm=algorith
                                        )

Ahora tenemos que elegir en cada uno cual ha sido las mejores opciones 
* En  **Cancer** : 
 * Algoritmo:auto
 * Numero de vecinos:7 
 * Peso: uniforme
* En **diabetes**: 
 * Algoritmo: auto
 * Numero de vecinos: 23
 * Peso:Distancia
* En **Titanic**:
 * Algoritmo: brute
 * Numero de vecinos: 7
 * Peso:Distancia

Ahora describiremos cada elemento que hemos usado en la optimizacion:
* Weights: es la forma en la que le da pesos a las distintas prediccianes puede ser o **uniforme** (todos pesan lo mismo) y **distance** (el peso es determinado por la distancia)
* Numero de vecinos: es el numero de vecinos que tendriamos en cuenta 
* Algoritmo: es el tipo de algoritmo que usaria hemos escogido brute y auto porque son por asi decirlos totalmente opuestos


## 5.2Arboles de decision

### 5.2.1 Cancer Wisconsin

In [ ]:
aux=pasosC.copy()
aux.append(('tree',clone(decision_tree_model)))
pipe = Pipeline(steps=aux)

criterion = ["gini", "entropy"]
max_depth=np.arange(5,25,10)
min_samples_leaf= np.arange(5,50,15)
ccp_alpha = [0, 0.001, 0.0025,0.005]

decision_tree_clfC = optimize_params3(pipe,
                                        CX_train,CYT, cv,
                                        scoring=scoring,
                                        tree__criterion = criterion,
                                        tree__max_depth = max_depth,
                                        tree__min_samples_leaf= min_samples_leaf,
                                        tree__ccp_alpha = ccp_alpha)

In [ ]:
aux=pasosD.copy()
aux.append(('tree',clone(decision_tree_model)))
pipe = Pipeline(steps=aux)

criterion = ["gini", "entropy"]
decision_tree_clfD = optimize_params3(pipe,
                                        DX_train,Dy_train, cv,
                                        scoring=scoring,
                                        tree__criterion = criterion,
                                        tree__max_depth = max_depth,
                                        tree__min_samples_leaf= min_samples_leaf,
                                        tree__ccp_alpha = ccp_alpha)

In [ ]:
aux=pasosT.copy()
aux.append(('tree',clone(decision_tree_model)))
pipe = Pipeline(steps=aux)

criterion = ["gini", "entropy"]


decision_tree_clfT = optimize_params3(pipe,
                                        TX_train,Ty_train, cv,
                                        scoring=scoring,
                                        tree__criterion = criterion,
                                        tree__max_depth = max_depth,
                                        tree__min_samples_leaf= min_samples_leaf,
                                        tree__ccp_alpha = ccp_alpha)

Los mejores resultados han sido:
* Cancer:
 * Ccp_alpha:0.005
 * Profundidad max:5
 * Criterio:Entropia
 * Min samples leaf:20
* Diabetes:
 * Ccp_alpha:0.005
 * Profundidad max:5
 * Criterio:gini
 * Min samples leaf:20
* Titanic:
 * Ccp_alpha:0
 * Profundidad max:5
 * Criterio:gini
 * Min samples leaf:20

A continuacion explicaremos los distintos  criterios:
* CCP alpha: determina el grado de penalizacion 
* Profundidad:se refiere a la profundidad maxima del arbol
* Criterion: es el tipo de cadena y puede ser gini o entropy siendo el primero para la impureza y el segundo para la ganancia 
* Ejemplos minimos es el minimo de observaciones que debe tener 

## 5.3 Adaptive Boosting (AdaBoost)

### 5.3.1 Cancer Wilsconsin

In [ ]:

#scoring=['recall','precision']
#scoring = (make_scorer(recall_score, pos_label="1"))

aux=pasosC.copy()
aux.append(('adaboostclassifier',AdaBoostClassifier(random_state=random_state)))
pipe = Pipeline(steps=aux)
base_estimator = [DecisionTreeClassifier(random_state=random_state)]
learning_rate = [0.95, 1.0]
criterion = ["gini", "entropy"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.015]
n_estimators = np.arange(20,85,20)
n_bins=[2,3]

adaboost_clfC = optimize_params3(pipe,
                                     CX_train,CYT, cv,
                                     scoring=scoring,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__n_estimators=n_estimators,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha,
                                     #discretizer__n_bins=n_bins
                                )

In [ ]:
base_estimator = clone(decision_tree_model)

base_estimator = [base_estimator]
aux=pasosD.copy()
aux.append(('adaboostclassifier',AdaBoostClassifier(random_state=random_state)))
pipe = Pipeline(steps=aux)
adaboost_clfD = optimize_params3(pipe,
                                DX_train,Dy_train, cv,
                                scoring=scoring,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__n_estimators=n_estimators,

                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                    
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)
pipe.get_params().keys()

In [ ]:

aux=pasosT.copy()
aux.append(('adaboostclassifier',AdaBoostClassifier(random_state=random_state)))
pipe = Pipeline(steps=aux)
adaboost_clfT = optimize_params3(pipe,
                                     TX_train,Ty_train, cv,scoring=scoring,
                                #refit=refit,
                                    adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__n_estimators=n_estimators,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha)

* Cancer:
 * Ccp alpha:0
 * Criterion:gini
 * Max depth:1
 * Learning rate:1.0
 * Numero de estimadores:20
* Diabetes:
 * Ccp alpha:0.015
 * Criterion:entropia
 * Max depth:2
 * Learning rate:0.95
 * Numero de estimadores:20
* Titanic:
 * Ccp alpha:0.015
 * Criterion:gini
 * Max depth:1
 * Learning rate:1
 * Numero de estimadores:20

* CCP alpha: determina el grado de penalizacion
* Criterion: es el tipo de cadena y puede ser gini o entropy siendo el primero para la impureza y el segundo para la ganancia 
* n estimator: numero maximo de estimadores en los que finaliza el boosting

## 5.4 Bootstrap Aggregating (Bagging)

### 5.4.1 Cancer Wisconsin

In [ ]:
aux=pasosC.copy()
aux.append(('bagging',clone(BaggingClassifier(random_state=random_state))))
pipe = Pipeline(steps=aux)
criterion = ["gini", "entropy"]
max_depth = np.arange(1,6,1)
ccp_alpha = [0.0, 0.015]
n_estimators = np.arange(5,25,5)
n_bins=[2,3]
#scoring = {'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label=1)}

bagging_clfC = optimize_params3(pipe,
                                    CX_train,CYT, cv,
                                    scoring=scoring,
                                    bagging__base_estimator=base_estimator,
                                    bagging__n_estimators=n_estimators,
                                    bagging__base_estimator__criterion=criterion,
                                    bagging__base_estimator__max_depth=max_depth,
                                    bagging__base_estimator__ccp_alpha=ccp_alpha,
                                    #discretizer__n_bins=n_bins
                              )

In [ ]:
aux=pasosD.copy()
aux.append(('bagging',clone(BaggingClassifier(random_state=random_state))))
pipe = Pipeline(steps=aux)
#scoring = {'AUC':'roc_auc', 'Recall':make_scorer(recall_score, pos_label=1)}

bagging_clfD = optimize_params3(pipe,
                                    DX_train,Dy_train, cv,
                                    scoring=scoring,
                                    bagging__base_estimator=base_estimator,
                                    bagging__n_estimators=n_estimators,
                                    bagging__base_estimator__criterion=criterion,
                                    bagging__base_estimator__max_depth=max_depth,
                                    bagging__base_estimator__ccp_alpha=ccp_alpha
                                    #discretizer__n_bins=n_bins
                              )

In [ ]:
aux=pasosT.copy()
aux.append(('bagging',clone(BaggingClassifier(random_state=random_state))))
pipe = Pipeline(steps=aux)


bagging_clfT = optimize_params3(pipe,
                                    TX_train,Ty_train, cv,
                                    scoring=scoring,
                                    bagging__base_estimator=base_estimator,
                                    bagging__n_estimators=n_estimators,
                                    bagging__base_estimator__criterion=criterion,
                                    bagging__base_estimator__max_depth=max_depth,
                                    bagging__base_estimator__ccp_alpha=ccp_alpha,
                                    #discretizer__n_bins=n_bins
                              )

* Cancer:
 * Cpp alpha:0
 * Criterion:entropy
 * Max depth:2
 * Numero de estimadores:20
* Diabetes:
 * Cpp alpha:0
 * Criterion:entropy
 * Max depth:4
 * Numero de estimadores:20
* Titanic:
 * Cpp alpha:0
 * Criterion:gini
 * Max depth:1
 * Numero de estimadores:15

* CCP alpha: determina el grado de penalizacion
* Criterion: es el tipo de cadena y puede ser gini o entropy siendo el primero para la impureza y el segundo para la ganancia
* Numero de estimadores de base


## 5.5 Random Forest

### 5.5.1 Cancer Wisconsin

In [ ]:
aux=pasosC.copy()
aux.append(('forest',clone(random_forest_model)))
pipe = Pipeline(steps=aux)
criterion = ["gini", "entropy"]
max_features = ["sqrt"]
n_estimators = np.arange(50,200,75)
max_depth= np.arange(1,7,1)
ccp_alpha=[0.0, 0.015]
#n_bins=[2,3]

random_forest_clfC = optimize_params3(pipe,
                                          CX_train,CYT, cv,
                                          scoring=scoring,
                                          forest__criterion=criterion,
                                          forest__max_features=max_features,
                                          forest__n_estimators=n_estimators,
                                          forest__max_depth=max_depth,
                                          forest__ccp_alpha=ccp_alpha
                                          )
print(pipe.get_params().keys())

In [ ]:
aux=pasosD.copy()
aux.append(('forest',clone(random_forest_model)))
pipe = Pipeline(steps=aux)
random_forest_clfD = optimize_params3(pipe,
                                          DX_train,Dy_train, cv,
                                          scoring=scoring,
                                          forest__criterion=criterion,
                                          forest__max_features=max_features,
                                          forest__n_estimators=n_estimators,
                                          forest__max_depth=max_depth,
                                          forest__ccp_alpha=ccp_alpha,
                                          #discretizer__n_bins=n_bins
                                    )

In [ ]:
aux=pasosT.copy()
aux.append(('forest',clone(random_forest_model)))
pipe = Pipeline(steps=aux)
random_forest_clfT =optimize_params3(pipe,
                                          TX_train,Ty_train, cv,
                                          scoring=scoring,
                                          forest__criterion=criterion,
                                          forest__max_features=max_features,
                                          forest__n_estimators=n_estimators,
                                          forest__max_depth=max_depth,
                                          forest__ccp_alpha=ccp_alpha,
                                          #discretizer__n_bins=n_bins
                                    )

* Cancer:
 * CCp alpha:0
 * Criterion:gini
 * Max depth:5
 * Max features:qrt
 * Numero de estimadores:125
* Diabetes
 * CCp alpha:0
 * Criterion:entropy
 * Max depth:6
 * Max features:sqrt
 * Numero de estimadores:50
* Titanic
 * CCp alpha:0.015
 * Criterion:gini
 * Max depth:1
 * Max features:sqrt
 * Numero de estimadores:50

* CCP alpha: determina el grado de penalizacion
* Criterion: es el tipo de cadena y puede ser gini o entropy siendo el primero para la impureza y el segundo para la ganancia 
* Max depth profundidad maxima del arbol
* Max features:es la caracteristicas para clasificar
* Numero de estimadores: el numero de arboles que va a tener 

## 5.6 Gradient Tree Bosting (Gradient Boosting)


### 5.6.1 Cancer Wisconsin

In [ ]:
aux=pasosC.copy()
aux.append(('gBoosting',clone(gradient_boosting_model)))
pipe = Pipeline(steps=aux)
learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = np.arange(1,5,1)
ccp_alpha = [0.0, 0.015]
n_estimators = np.arange(50,200,50)

gradient_boosting_clfC =optimize_params3(pipe,
                                              CX_train,CYT, cv,
                                              scoring=scoring,
                                              gBoosting__learning_rate=learning_rate,
                                              gBoosting__criterion=criterion,
                                              gBoosting__max_depth=max_depth,
                                              gBoosting__ccp_alpha=ccp_alpha,
                                              gBoosting__n_estimators=n_estimators)

In [ ]:
aux=pasosD.copy()
aux.append(('gBoosting',clone(gradient_boosting_model)))
pipe = Pipeline(steps=aux)

gradient_boosting_clfD = optimize_params3(pipe,
                                              DX_train,Dy_train, cv,
                                              scoring=scoring,
                                              gBoosting__learning_rate=learning_rate,
                                              gBoosting__criterion=criterion,
                                              gBoosting__max_depth=max_depth,
                                              gBoosting__ccp_alpha=ccp_alpha,
                                              gBoosting__n_estimators=n_estimators)

In [ ]:
aux=pasosT.copy()
aux.append(('gBoosting',clone(gradient_boosting_model)))
pipe = Pipeline(steps=aux)
gradient_boosting_clfT = optimize_params3(pipe,
                                              TX_train,Ty_train, cv,
                                              scoring=scoring,
                                              gBoosting__learning_rate=learning_rate,
                                              gBoosting__criterion=criterion,
                                              gBoosting__max_depth=max_depth,
                                              gBoosting__ccp_alpha=ccp_alpha,
                                              gBoosting__n_estimators=n_estimators
                                        )
                                         

* Cancer
 * ccp alpha:0
 * criterion:friendman_mse
 * Learning rate:0.01
 * max depth:2
 * n estimator:150
* Diabetes
 * ccp alpha:0
 * criterion:friendman_mse
 * Learning rate:0.05
 * max depth:4
 * n estimator:100
* Titanic:
 * ccp alpha:0
 * criterion:friedman_mse
 * Learning rate:0.
 * max depth:1
 * n estimator:

* CCP alpha: determina el grado de penalizacion
* Criterion: es el tipo de cadena y puede ser gini o entropy siendo el primero para la impureza y el segundo para la ganancia 
* learning rate:sirve para reducir la contribucion de cada arbol
* Numero de estimadores 

## 5.7 Histogram-Based Gradient Boosting (Histogram Gradient Boosting)


### 5.7.1 Cancer Wisconsin

In [ ]:

aux=pasosC.copy()
aux.append(('histGBoosting',clone(hist_gradient_boosting_model)))
pipe = Pipeline(steps=aux)
learning_rate = np.arange(0.01,0.075,0.02)
max_leaf_nodes = np.arange(15,125,40)#40
max_depth = np.arange(5,25,2)

hist_gradient_boosting_clfC = optimize_params3(pipe,
                                                  CX_train,CYT, cv,scoring=scoring,
                                                   histGBoosting__learning_rate=learning_rate,
                                                   histGBoosting__max_leaf_nodes=max_leaf_nodes,
                                                   histGBoosting__max_depth=max_depth)

In [ ]:

aux=pasosD.copy()
aux.append(('histGBoosting',clone(hist_gradient_boosting_model)))
pipe = Pipeline(steps=aux)

hist_gradient_boosting_clfD = optimize_params3(pipe,
                                                   DX_train,Dy_train, cv,scoring=scoring,
                                                   histGBoosting__learning_rate=learning_rate,
                                                   histGBoosting__max_leaf_nodes=max_leaf_nodes,
                                                   histGBoosting__max_depth=max_depth)

In [ ]:

aux=pasosT.copy()
aux.append(('histGBoosting',clone(hist_gradient_boosting_model)))
pipe = Pipeline(steps=aux)

hist_gradient_boosting_clfT = optimize_params3(pipe,
                                                   TX_train,Ty_train, cv,scoring=scoring,
                                                   histGBoosting__learning_rate=learning_rate,
                                                   histGBoosting__max_leaf_nodes=max_leaf_nodes,
                                                   histGBoosting__max_depth=max_depth)

* Cancer:
 * Learning rate:0.04999
 * Max depth:5
 * Max leaf nodes:15
* Diabetes:
 * Learning rate:0.049999
 * Max depth:7
 * Max leaf nodes:15
* Titanic:
 * Learning rate:0.0499
 * Max depth:5
 * Max leaf nodes:15

# 5. Modelo final

In [ ]:
estimatorC={
    "Nearest neighborsC": k_neighbors_clfC,
    "Decision treeC": decision_tree_clfC,
    "AdaBoostC": adaboost_clfC,
    "BaggingC": bagging_clfC,
    "Random ForestsC": random_forest_clfC,
    "Gradient BoostingC": gradient_boosting_clfC,
    "Histogram Gradient BoostingC": hist_gradient_boosting_clfC 
}
estimatorD={
    "Nearest neighborsD": k_neighbors_clfD,
    "Decision treeD": decision_tree_clfD,
    "AdaBoostD": adaboost_clfD,
    "BaggingD": bagging_clfD,
    "Random ForestsD": random_forest_clfD,
    "Gradient BoostingD": gradient_boosting_clfD,
    "Histogram Gradient BoostingD": hist_gradient_boosting_clfD
}
estimatorT={
    "Nearest neighborsT": k_neighbors_clfT,
    "Decision treeT": decision_tree_clfT,
    "AdaBoostT": adaboost_clfT,
    "BaggingT": bagging_clfT,
    "Random ForestsT": random_forest_clfT,
    "Gradient BoostingT": gradient_boosting_clfT,
    "Histogram Gradient BoostingT": hist_gradient_boosting_clfT 
}

Empezamos primero creando copias de los test

In [ ]:
CYTest=Cy_test.replace({"B":1,"M":0}).copy()
CXTest=CX_test.copy()
DYTest=Dy_test.copy()
DXTest=DX_test.copy()
TYTest=Ty_test.copy()
TXTest=TX_test.copy()

In [ ]:
def evaluate_estimators2(estimators, X, y, pos_label):
    columns = ["roc_auc_score", "recall_score", "mean"]
    results = pd.DataFrame(columns=columns)

    for estimator in estimators:
        y_pred=estimators[estimator].predict(X)
        
        recall = recall_score(y, y_pred, pos_label=pos_label)
        
        if pos_label=="M" or pos_label=="B":
            y_pred_aux=[1 if x=="M" else 0 for x in y_pred]
            y_aux=[1 if x=="M" else 0 for x in y]
            roc=roc_auc_score(y_aux, y_pred_aux) 
        else:
            roc=roc_auc_score(y, y_pred)
        
        mean=(recall+roc)/2
        results.loc[estimator] = [roc,recall,mean]

    return results

# Analisis de kernel 


Para el análisis del [kernel](https://www.kaggle.com/shivan118/student-performance-analisys#-Data-Preprocessing) hemos seleccionado un estudio bastante extenso en el que nos hemos centrado en destacar lo que consideramos que son errores o parte que nosotros hubiéramos hecho de forma distinta.

El database usado es una colección de datos de estudiantes que cuenta con datos personales y registros académicos con el objetivo de hacer un estudio sobre el rendimiento escolar.

En este kernel sobre las notas de los estudiantes de los distintos grupos vemos que tiene dos partes al igual que la nuestra una de exploración y otra de preprocesamiento

Nada más empezar, tras la carga de datos, vemos un fallo que es no comprobar las tuplas de manera aleatoria sino que mira la cabeza y la cola del dataframe.

El database en cuestión tiene variables numéricas y las variables no numéricas para estas ultima le pone más gráficos que nosotros a las nuestras por lo demás es muy parecido. En lugar de ver los datos anómalos como hacemos nosotros con un diagrama de caja este lo hace con la grafica 

En la parte de preprocesamiento vemos que  lo primero que hace es remplazar las distintas variables no numéricas por números(como hicimos con el género de Titanic)

Al acabar el preprocesamiento hace la división entre train y test, cuando lo recomendable es hacerlo al principio para no tocar en nada al test hasta la evaluación.

Nosotros habríamos eliminamos la categoría “total score” o “porcentaje” ya que son idénticas entre ellas como se puede ver en el mapa de calor.

Luego hace una predicción la cual da un acierto del 100% lo cual nos hace pensar en sobreajuste. 
La conclusión es que el estudio de este database es aceptable pero luego el preprocesamiento, lo único que hace es cambiar de alfanuméricas a numéricas por otro lado el problema es que el database no puede dar a mucho mas 


In [ ]:
#utils.evaluate_estimators(estimatorC,CXTest,CYTest)